In [ ]:
import cv2
import face_recognition as fr
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk, ImageDraw

In [ ]:
class Application:

    def __init__(self, root):
        self.root=root
        self.root.title("Face Detection and Recognition System")
        self.root.configure(background='blue')

        self.l1=tk.Label(root, text="Face Detection and Recognition System", bg="YELLOW", relief="ridge", fg="RED", font=("Times", 16), width=60)
        self.l1.grid(row=1, column=1, columnspan=2, sticky="W", padx=10, pady=10)

        self.l2=tk.Label(root, text="Press this button for Face Detection in an Image ==>", bg="YELLOW", relief="ridge", fg="RED", font=("Times", 13), width=50)
        self.l2.grid(row=2,column=1, sticky="W", padx=10, pady=10)

        self.l3=tk.Label(root, text="Press this button for Face Recognition in two Images ==>", bg="YELLOW", relief="ridge", fg="RED", font=("Times", 13), width=50)
        self.l3.grid(row=3, column=1, sticky="W", padx=10, pady=10)

        self.known_image=None
        self.known_face_encoding=None

        def faceDetectWindow():
            fdwin=tk.Toplevel(self.root)
            fdwin.title("Face Detection System")
            fdwin.configure(background='orange')

            l4=tk.Label(fdwin, text="Face Detection System", bg="BLUE", relief="ridge", fg="YELLOW", font=("Times", 16), width=60)
            l4.grid(row=1, column=2, sticky="W", padx=10, pady=10)

            face_cap=cv2.CascadeClassifier("D:/Python 3.13.1/Lib/site-packages/cv2/data/haarcascade_frontalface_default.xml")
            
            def upload_image():
                file_path=filedialog.askopenfilename()
                if file_path:
                    image=cv2.imread(file_path)
                    new_image=cv2.resize(image, (0,0), fx=0.2, fy=0.2)
                    while True:
                        col=cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)
                        faces=face_cap.detectMultiScale(col, scaleFactor=1.1, minNeighbors=5, minSize=(30,30), flags=cv2.CASCADE_SCALE_IMAGE)
                        for (x,y,w,h) in faces:
                            cv2.rectangle(new_image, (x,y), (x+w,y+h), (0,255,0), 2)
                        cv2.imshow("Image Capture", new_image)
                        if cv2.waitKey(0):
                            cv2.destroyAllWindows()
                            fdwin.destroy()
                            break

            b4=tk.Button(fdwin, text="Upload an Image to detect face", command=upload_image)
            b4.grid(row=2, column=2, padx=10, pady=10)

            b5=tk.Button(fdwin, text="   Close   ", command=fdwin.destroy)
            b5.grid(row=4, column=1, columnspan=2, padx=10, pady=10)
            
            fdwin.mainloop()

        def faceRecognizeWindow():
            frwin=tk.Toplevel(self.root)
            frwin.title("Face Recognition System")
            frwin.configure(background='orange')

            l6=tk.Label(frwin, text="Face Recognition System", bg="BLUE", relief="ridge", fg="YELLOW", font=("Times", 16), width=60)
            l6.grid(row=1, column=1, columnspan=2, padx=10, pady=10)

            result_frame=tk.Frame(frwin, bg='orange')
            result_frame.grid(row=2, column=1, columnspan=2, padx=10, pady=10)

            def mark_face(image, face_location, status):
                draw=ImageDraw.Draw(image)
                top, right, bottom, left=face_location
                color="red" if status=="no_match" else "green"
                draw.rectangle(((left, top), (right, bottom)), outline=color, width=2)

            def upload_known_image():
                file_path=filedialog.askopenfilename()
                if file_path:
                    self.known_image=fr.load_image_file(file_path)
                    encodings=fr.face_encodings(self.known_image)
                    if encodings:
                        self.known_face_encoding=encodings[0]
                        pil_img=Image.fromarray(self.known_image).resize((200, 200), Image.LANCZOS)
                        tk_img=ImageTk.PhotoImage(pil_img)
                        l7=tk.Label(result_frame, image=tk_img)
                        l7.grid(row=2, column=1, padx=10, pady=10)
                        l7.image=tk_img
                    else:
                        print("No face found in the image.")

            def upload_for_recognition():
                if self.known_face_encoding is None:
                    messagebox.showerror("Error", "Please upload a known image first.")
                    return

                file_path=filedialog.askopenfilename()
                if file_path:
                    unknown_image=fr.load_image_file(file_path)
                    face_locations=fr.face_locations(unknown_image)
                    unknown_face_encodings=fr.face_encodings(unknown_image, face_locations)

                    is_matched=False
                    pil_unknown_image=Image.fromarray(unknown_image)
                    for face_encoding, face_location in zip(unknown_face_encodings, face_locations):
                        results=fr.compare_faces([self.known_face_encoding], face_encoding)
                        status="match" if results[0] else "no_match"
                        mark_face(pil_unknown_image, face_location, status)
                        if results[0]:
                            is_matched=True

                    tk_img2=ImageTk.PhotoImage(pil_unknown_image.resize((200, 200), Image.LANCZOS))
                    l8=tk.Label(result_frame, image=tk_img2)
                    l8.grid(row=2, column=2, padx=10, pady=10)
                    l8.image=tk_img2
                    
                    if is_matched:
                        l9=tk.Label(frwin, text="It's A Match!", bg="green", relief="ridge", fg="blue", font=("Times", 14), width=50)
                        l9.grid(row=3, column=1, columnspan=2, padx=10, pady=10)
                    else:
                        l9=tk.Label(frwin, text="No Match Found!", bg="red", relief="ridge", fg="blue", font=("Times", 14), width=50)
                        l9.grid(row=3, column=1, columnspan=2, padx=10, pady=10)

            b6=tk.Button(frwin, text="   Upload known Image   ", command=upload_known_image)
            b6.grid(row=4, column=1, padx=10, pady=10)

            b7=tk.Button(frwin, text="Upload unknown Image", command=upload_for_recognition)
            b7.grid(row=4, column=2, padx=10, pady=10)
            
            b8=tk.Button(frwin, text="   Close   ", command=frwin.destroy)
            b8.grid(row=5, column=1, columnspan=2, padx=10, pady=10)

            frwin.mainloop()

        self.b1=tk.Button(root, text="  Face Detection  ", command=faceDetectWindow)
        self.b1.grid(row=2, column=2, padx=10, pady=10)

        self.b2=tk.Button(root, text="Face Recognition", command=faceRecognizeWindow)
        self.b2.grid(row=3, column=2, padx=10, pady=10)

        self.b3=tk.Button(root, text="   Close   ", command=root.destroy)
        self.b3.grid(row=4, column=1, columnspan=2, padx=10, pady=10)

In [ ]:
if __name__=="__main__":
    root=tk.Tk()
    App=Application(root)
    root.mainloop()